# Fine-tune 对话摘要模型

![Pipeline](./img/generative_ai_pipeline_rlhf_plus.png)

<a name='1'></a>
## Set up Kernel and Required Dependencies

First, check that the correct kernel is chosen.

<img src="img/kernel_set_up.png" width="300"/>

You can click on that to see and check the details of the image, kernel, and instance type.

<img src="img/w3_kernel_and_instance_type.png" width="600"/>

In [2]:
import psutil

notebook_memory = psutil.virtual_memory()
print(notebook_memory)

if notebook_memory.total < 32 * 1000 * 1000 * 1000:
    print('*******************************************')    
    print('YOU ARE NOT USING THE CORRECT INSTANCE TYPE')
    print('PLEASE CHANGE INSTANCE TYPE TO  m5.2xlarge ')
    print('*******************************************')
else:
    correct_instance_type=True

svmem(total=33229983744, available=27538239488, percent=17.1, used=5322313728, free=436244480, active=4309553152, inactive=25311002624, buffers=1634304, cached=27469791232, shared=1236992, slab=2485346304)


In [3]:
%store -r model_checkpoint

In [4]:
try:
    model_checkpoint
except NameError:
    print("++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++")
    print("[ERROR] Please run the notebooks in the PREPARE section before you continue.")
    print("++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++")

In [5]:
print(model_checkpoint)

google/flan-t5-base


In [6]:
%store -r local_data_processed_path

In [7]:
try:
    local_data_processed_path
except NameError:
    print("++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++")
    print("[ERROR] Please run the notebooks in the PREPARE section before you continue.")
    print("++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++")

In [8]:
print(local_data_processed_path)

./data-summarization-processed/


# 加载包

In [9]:
# 导入transformers库中的几个重要模块
# AutoTokenizer：自动标记器，用于加载适合预训练模型的标记器
# AutoModelForSeq2SeqLM：用于加载适合序列到序列语言模型的预训练模型
# TrainingArguments：训练参数，用于设置训练过程中的各种参数
# Trainer：训练器，用于进行模型训练的类
# GenerationConfig：生成配置，用于设置模型生成文本的参数
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, TrainingArguments, Trainer, GenerationConfig

# 导入datasets库中的load_dataset函数，用于加载数据集
from datasets import load_dataset

# 导入datasets库，用于使用库中的其他功能
import datasets

# 导入PyTorch库，这是一个开源的深度学习平台
import torch

# 导入time模块，用于处理和格式化时间
import time

# 导入evaluate模块，这可能是一个用于模型评估的自定义模块
import evaluate

# 导入numpy库，这是一个用于处理大型多维数组和矩阵的库，提供了大量的数学函数来操作这些数组
import numpy as np

# 设置模型运行所使用的设备
# 如果有可用的CUDA设备（也就是GPU），就使用CUDA设备，否则就使用CPU
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

# 加载HuggingFace模型

我们可以直接从HuggingFace加载预训练的Flan-T5模型。请注意，我们将使用[base version](https://huggingface.co/google/flan-t5-base)的flan。此模型版本有大约2.47亿的模型参数，相比其他大型语言模型来说较小。如果想要获得更高质量的结果，我们建议研究此模型的更大版本。

In [10]:
# 使用AutoTokenizer的from_pretrained方法加载预训练模型对应的标记器
# model_checkpoint是预训练模型的名字或路径
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

# 使用AutoModelForSeq2SeqLM的from_pretrained方法加载预训练的序列到序列语言模型
# model_checkpoint是预训练模型的名字或路径
model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

In [11]:
# 使用列表推导式统计模型中的总参数数量
# p.numel()方法返回Tensor中元素的个数
params = sum(p.numel() for p in model.parameters())

# 打印模型中的总参数数量
print(f'Total Number of Model Parameters: {params}')

Total Number of Model Parameters: 247577856


# 加载处理后的数据

# 加载数据集

我们已经处理过的DialogSum数据集可以直接从我们的本地目录加载。这个数据集中有大约15k个对话示例，并且这些数据集都有相应的人工摘要。

In [12]:
# 使用load_dataset函数加载数据集
# local_data_processed_path是数据集的路径
# data_files是一个字典，键是数据集的名称（'train'、'test'、'validation'），值是对应的文件路径
# with_format("torch")方法用于将数据集格式化为PyTorch的Tensor格式
tokenized_dataset = load_dataset(
    local_data_processed_path,
    data_files={'train': 'train/*.parquet', 'test': 'test/*.parquet', 'validation': 'validation/*.parquet'}
).with_format("torch")

# 打印加载的数据集的内容
tokenized_dataset

Found cached dataset parquet (/root/.cache/huggingface/datasets/parquet/data-summarization-processed-501c81c8367fa59b/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)


  0%|          | 0/3 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'labels'],
        num_rows: 13014
    })
    test: Dataset({
        features: ['input_ids', 'labels'],
        num_rows: 723
    })
    validation: Dataset({
        features: ['input_ids', 'labels'],
        num_rows: 723
    })
})

# 在微调之前，用 Zero-Shot Prompts测试模型

在下面的示例中，我们强调了与数据集中提供的基线摘要相比，模型的摘要能力是不足的。你可以看到，与基线摘要相比，模型在总结对话时存在困难，但它确实从文本中提取出了一些重要信息，这表明可以对模型进行微调以完成手头的任务。

In [14]:
# 加载测试集中的一条数据，使用模型生成一个摘要，并将原始输入、模型生成的摘要和基线摘要打印出来。

# 设置要加载的数据的索引
idx = 2

# 使用tokenizer的decode方法将输入的标记解码为文本，skip_special_tokens=True表示跳过特殊标记
diag = tokenizer.decode(tokenized_dataset['test'][idx]['input_ids'], skip_special_tokens=True)

# 将解码后的文本重新编码为模型输入
model_input = tokenizer(diag, return_tensors="pt").input_ids

# 使用tokenizer的decode方法将标签的标记解码为文本，这是基线摘要
summary = tokenizer.decode(tokenized_dataset['test'][idx]['labels'], skip_special_tokens=True)

# 使用模型的generate方法生成摘要，设置最大新标记数量为200
original_outputs = model.to('cpu').generate(model_input, GenerationConfig(max_new_tokens=200))

# 将生成的摘要解码为文本
original_text_output = tokenizer.decode(original_outputs[0], skip_special_tokens=True)

# 将输入的文本中的'#'字符替换为'\n#'，用于格式化输出
diag_print = diag.replace(' #',' \n#')

# 打印原始输入
print(f"Prompt:\n--------------------------\n{diag_print}\n--------------------------")

# 打印模型生成的摘要
print(f'\nOriginal Model Response: {original_text_output}')

# 打印基线摘要
print(f'Baseline Summary : {summary}')

Prompt:
--------------------------
Summarize the following conversation. 
#Person1#: Hello. My name is John Sandals, and I've got a reservation. 
#Person2#: May I see some identification, sir, please? 
#Person1#: Sure. Here you are. 
#Person2#: Thank you so much. Have you got a credit card, Mr. Sandals? 
#Person1#: I sure do. How about American Express? 
#Person2#: Unfortunately, at the present time we take only MasterCard or VISA. 
#Person1#: No American Express? Okay, here's my VISA. 
#Person2#: Thank you, sir. You'll be in room 507, nonsmoking, with a queen-size bed. Do you approve, sir? 
#Person1#: Yeah, that'll be fine. 
#Person2#: That's great. This is your key, sir. If you need anything at all, anytime, just dial zero. Summary: 
--------------------------

Original Model Response: John Sandals has a reservation for a room at the Venetian Hotel in Las Vegas.
Baseline Summary : John Sandals has got a reservation. #Person1# asks for his identification and credit card and helps his 

# 微调模型

数据集已经预处理完毕，我们可以利用HuggingFace内置的 `Trainer` 类来微调我们的模型以适应手头的任务。请注意，全模型的训练在GPU上需要几个小时，所以为了节省时间，我们提供了一个已经在没有下采样的情况下训练了10个epoch的模型的检查点。如果你有时间尝试自己完全训练模型，请参考注释了解如何更改代码。如果你打算在GPU机器上训练，我们已经提供了一个 `ml.g5.xlarge` 实例的检查点作为起点

In [17]:
# 为了节省实验室的时间，我们将对我们的数据集进行子采样
# 如果你想花时间全面训练一个模型，可以自由修改这个子采样以创建一个更大的数据集
# lambda example, indice: indice % 100 == 0是一个匿名函数，该函数返回True如果索引值（indice）是100的整数倍，否则返回False
# filter函数将只保留那些索引值是100的倍数的样本，也就是说，每100个样本，只取一个
# with_indices=True参数指示filter函数在调用参数函数时传递样本的索引
sample_tokenized_dataset = tokenized_dataset.filter(lambda example, indice: indice % 100 == 0, with_indices=True)

# 设置模型的输出目录，名为“diag-summary-training-”后接当前时间戳
output_dir = f'./diag-summary-training-{str(int(time.time()))}'

# 设置训练参数
training_args = TrainingArguments(
    output_dir=output_dir,  # 指定模型输出的目录
    evaluation_strategy="epoch",  # 设置评估策略为每个epoch结束时进行一次评估
    save_strategy="epoch",  # 设置保存策略为每个epoch结束时保存一次模型
    learning_rate=1e-5,  # 设置学习率
    num_train_epochs=1,  # 设置训练的epoch数量
    # num_train_epochs=10, # 当你不在实验室并且有更多的时间去试验时，可以使用更高的epoch数量
    # per_device_train_batch_size和per_device_eval_batch_size是设定在训练和评估过程中每个设备上处理的数据批次（batch）的大小。
    # per_device_train_batch_size=4：这个参数设置了每个设备在训练过程中一次处理的样本数量为4。这意味着在每一步训练（一个步骤对应于一次参数更新）中，模型将基于4个样本的平均损失进行学习。选择合适的批次大小对于模型的训练很重要，因为它会影响模型的学习效率和训练的稳定性。
    # per_device_eval_batch_size=4：这个参数设置了在评估过程中每个设备一次处理的样本数量为4。在评估过程中，模型会计算这4个样本的平均损失或者其他评估指标，以估计模型在验证集上的性能。
    # 这两个参数的选择取决于你的硬件配置（主要是GPU内存）和数据集的大小。如果选择的批次太大，可能会导致内存溢出；如果批次太小，可能会影响训练的速度和模型的性能。所以在实际使用中，需要根据具体情况来调整这两个参数。
    per_device_train_batch_size=4,  # 设置每个设备的训练批次大小
    per_device_eval_batch_size=4,  # 设置每个设备的评估批次大小
    weight_decay=0.01,  # 设置权重衰减值
)

# 创建训练器
trainer = Trainer(
    model=model,  # 指定模型
    args=training_args,  # 指定训练参数
    train_dataset=sample_tokenized_dataset['train'],  # 指定训练数据集
    eval_dataset=sample_tokenized_dataset['validation']  # 指定评估数据集
)



Filter:   0%|          | 0/13014 [00:00<?, ? examples/s]

Filter:   0%|          | 0/723 [00:00<?, ? examples/s]

Filter:   0%|          | 0/723 [00:00<?, ? examples/s]

In [18]:
# 开始训练模型
trainer.train()

/opt/conda/lib/python3.7/site-packages/transformers/optimization.py:395: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


Epoch,Training Loss,Validation Loss
1,No log,35.455990


TrainOutput(global_step=33, training_loss=38.12038722182765, metrics={'train_runtime': 451.36, 'train_samples_per_second': 0.29, 'train_steps_per_second': 0.073, 'total_flos': 89703213170688.0, 'train_loss': 38.12038722182765, 'epoch': 1.0})

# 加载训练过的模型和原始模型

模型完成训练后，我们加载来自HuggingFace的原始模型和经过微调的模型，进行一些比较。

In [19]:
# 从Amazon S3复制模型检查点到本地
!aws s3 cp --recursive s3://dsoaws/models/flan-dialogue-summary-checkpoint/ ./flan-dialogue-summary-checkpoint/

download: s3://dsoaws/models/flan-dialogue-summary-checkpoint/generation_config.json to flan-dialogue-summary-checkpoint/generation_config.json
download: s3://dsoaws/models/flan-dialogue-summary-checkpoint/config.json to flan-dialogue-summary-checkpoint/config.json
download: s3://dsoaws/models/flan-dialogue-summary-checkpoint/scheduler.pt to flan-dialogue-summary-checkpoint/scheduler.pt
download: s3://dsoaws/models/flan-dialogue-summary-checkpoint/rng_state.pth to flan-dialogue-summary-checkpoint/rng_state.pth
download: s3://dsoaws/models/flan-dialogue-summary-checkpoint/trainer_state.json to flan-dialogue-summary-checkpoint/trainer_state.json
download: s3://dsoaws/models/flan-dialogue-summary-checkpoint/training_args.bin to flan-dialogue-summary-checkpoint/training_args.bin
download: s3://dsoaws/models/flan-dialogue-summary-checkpoint/pytorch_model.bin to flan-dialogue-summary-checkpoint/pytorch_model.bin
download: s3://dsoaws/models/flan-dialogue-summary-checkpoint/optimizer.pt to fl

In [20]:
# 如果你有自己训练的模型，想与我们的模型进行对比，可以修改以下代码行
# 使其包含你的检查点目录

supervised_fine_tuned_model_path = "./flan-dialogue-summary-checkpoint"
# supervised_fine_tuned_model_path = f"./{output_dir}/<put-your-checkpoint-dir-here>"

# 从预先训练好的模型路径加载微调过的模型
tuned_model = AutoModelForSeq2SeqLM.from_pretrained(supervised_fine_tuned_model_path)

# 从预训练模型的检查点加载模型
model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

In [21]:
%store supervised_fine_tuned_model_path

Stored 'supervised_fine_tuned_model_path' (str)


# 微调后的 Zero Shot 推理

像许多GenAI应用一样，采用定性方法，问自己这样的问题："我的模型是否按照预期的方式运行？"通常是一个好的起点。在下面的例子中（我们用这个例子开始了这个笔记本），你可以看到经过微调的模型能够创建一个合理的对话摘要，相比之下，原始模型无法理解对模型的要求。

In [22]:
# 设置要测试的样本的索引
idx = 2

# 使用tokenizer解码测试集中该索引对应的输入数据，并跳过特殊的标记（如：[CLS], [SEP]）
diag = tokenizer.decode(tokenized_dataset['test'][idx]['input_ids'], skip_special_tokens=True)

# 使用tokenizer将解码后的输入数据编码为模型可以接收的格式
model_input = tokenizer(diag, return_tensors="pt").input_ids

# 使用tokenizer解码测试集中该索引对应的标签数据（即：真实的摘要），并跳过特殊标记
summary = tokenizer.decode(tokenized_dataset['test'][idx]['labels'], skip_special_tokens=True)

# 使用原始模型生成摘要，限制最多生成200个新的标记，并使用束搜索来提高生成质量
original_outputs = model.to('cpu').generate(
    model_input,
    GenerationConfig(max_new_tokens=200, num_beams=1),
)

# 使用微调过的模型生成摘要，限制最多生成200个新的标记，并使用束搜索来提高生成质量
outputs = tuned_model.to('cpu').generate(
    model_input,
    GenerationConfig(max_new_tokens=200, num_beams=1,),
)

# 使用tokenizer解码生成的摘要，并跳过特殊标记
text_output = tokenizer.decode(outputs[0], skip_special_tokens=True)

# 将输入数据中的 '#' 替换为 '\n#'，用于更美观地打印
diag_print = diag.replace(' #',' \n#')

# 打印输入数据、原始模型和微调模型生成的摘要，以及原始数据集中的真实摘要
print(f"Prompt:\n--------------------------\n{diag_print}\n--------------------------")
print(f'Flan-T5 response: {original_text_output}')
print(f'Our instruct-tuned response (on top of Flan-T5): {text_output}')
print(f'Baseline summary from original dataset: {summary}')

Prompt:
--------------------------
Summarize the following conversation. 
#Person1#: Hello. My name is John Sandals, and I've got a reservation. 
#Person2#: May I see some identification, sir, please? 
#Person1#: Sure. Here you are. 
#Person2#: Thank you so much. Have you got a credit card, Mr. Sandals? 
#Person1#: I sure do. How about American Express? 
#Person2#: Unfortunately, at the present time we take only MasterCard or VISA. 
#Person1#: No American Express? Okay, here's my VISA. 
#Person2#: Thank you, sir. You'll be in room 507, nonsmoking, with a queen-size bed. Do you approve, sir? 
#Person1#: Yeah, that'll be fine. 
#Person2#: That's great. This is your key, sir. If you need anything at all, anytime, just dial zero. Summary: 
--------------------------
Flan-T5 response: John Sandals has a reservation for a room at the Venetian Hotel in Las Vegas.
Our instruct-tuned response (on top of Flan-T5): John Sandals has a reservation and checks in with his VISA. #Person2# helps him to

# 用ROUGE指标进行的定量结果

[ROUGE metric](https://en.wikipedia.org/wiki/ROUGE_(metric)) 有助于量化模型产生的摘要的有效性。它将摘要与通常由人类创建的"基线"摘要进行比较。虽然不完美，但它确实给出了我们通过微调所取得的摘要效果整体提升的一种指示。

In [25]:
# 加载 "rouge" 评估工具。这里的 "rouge" 指的是一种用于评估自动文本摘要质量的指标。它通过比较生成的摘要和参考摘要之间的重叠来计算得分。
# 加载评估指标 "rouge"
rouge = evaluate.load('rouge')

下面是一些例子来说明ROUGE评估的工作方式：

假设我们有一个参考摘要（真实摘要）和一个生成摘要（由模型生成）。

参考摘要：
```
The cat sat on the mat.
```
生成摘要：
```
The cat is sitting on the mat.
```
如果我们计算ROUGE-1（即计算一元组的重叠），我们看到这两个摘要有5个共享的词（"The", "cat", "on", "the", "mat"）。所以ROUGE-1的召回率就是5/6=0.83，精确率就是5/7=0.71，F1得分是2*(0.83*0.71)/(0.83+0.71)=0.77。

如果我们计算ROUGE-2（即计算二元组的重叠），共享的二元组有3个（"The cat", "on the", "the mat"），所以ROUGE-2的召回率就是3/5=0.6，精确率就是3/6=0.5，F1得分是2*(0.6*0.5)/(0.6+0.5)=0.54。

注意，ROUGE通常更关注召回率而不是精确率，因为在摘要任务中，我们更关心生成的摘要能覆盖多少参考摘要的信息。

另外，还有一些其他的ROUGE评估方法，例如ROUGE-L，它计算的是最长公共子序列（LCS），和ROUGE-S，它计算的是跳跃二元组（skip-bigram）

## 评估部分摘要

In [23]:
# 由于时间原因，我们只生成每个模型的少数几个摘要
# 在实验之外，一个好的练习是增加生成的验证摘要的数量
# 从测试数据集中选择前10条对话
dialogues = tokenized_dataset['test'][0:10]['input_ids']
# 选择相应的摘要作为基线摘要
baseline_summaries = tokenized_dataset['test'][0:10]['labels']

# 解码原始摘要
human_baseline_summaries = []
# 对于每个基线摘要，使用tokenizer进行解码，并忽略特殊标记
for base_summary in baseline_summaries:
    human_baseline_summaries.append(tokenizer.decode(base_summary, skip_special_tokens=True))

# 生成摘要
# 使用原始模型生成摘要，每个摘要的最大长度设为200个标记
original_outputs = model.generate(dialogues, GenerationConfig(max_new_tokens=200))
# 使用调优后的模型生成摘要，每个摘要的最大长度设为200个标记
tuned_outputs = tuned_model.generate(dialogues, GenerationConfig(max_new_tokens=200))

In [26]:
# 在列表中存储摘要
original_model_summaries = []  # 存储原始模型生成的摘要
tuned_model_summaries = []  # 存储调优后的模型生成的摘要

# 解码所有摘要
# 使用zip函数同时遍历原始模型和调优模型生成的摘要
for original_summary, tuned_summary in zip(original_outputs, tuned_outputs):
    # 使用tokenizer对原始模型生成的摘要进行解码，忽略特殊标记，并添加到对应的列表中
    original_model_summaries.append(tokenizer.decode(original_summary, skip_special_tokens=True))
    # 使用tokenizer对调优模型生成的摘要进行解码，忽略特殊标记，并添加到对应的列表中
    tuned_model_summaries.append(tokenizer.decode(tuned_summary, skip_special_tokens=True))

In [27]:
# 使用ROUGE评估指标计算原始模型生成的摘要与人类生成的摘要之间的相似度
original_results = rouge.compute(
    # 预测的摘要，即原始模型生成的摘要
    predictions=original_model_summaries,
    # 参考的摘要，即人类生成的摘要
    references=human_baseline_summaries,
    # 使用聚合器，这样可以计算多个摘要的平均ROUGE得分
    use_aggregator=True,
    # 使用词干处理，这样可以将单词的各种形态（如复数、过去式等）统一为其基本形式进行比较
    use_stemmer=True,
)

In [29]:
# 使用 ROUGE 评估指标计算调优后的模型生成的摘要与人类生成的摘要之间的相似度
tuned_results = rouge.compute(
    # 预测的摘要，即调优后的模型生成的摘要
    predictions=tuned_model_summaries,
    # 参考的摘要，即人类生成的摘要
    references=human_baseline_summaries,
    # 使用聚合器，这样可以计算多个摘要的平均 ROUGE 得分
    use_aggregator=True,
    # 使用词干处理，这样可以将单词的各种形态（如复数、过去式等）统一为其基本形式进行比较
    use_stemmer=True,
)

In [30]:
original_results

{'rouge1': 0.23899693678641046,
 'rouge2': 0.08932806324110672,
 'rougeL': 0.21776705653021444,
 'rougeLsum': 0.21325132275132275}

In [31]:
tuned_results

{'rouge1': 0.5243400833160587,
 'rouge2': 0.240785255433749,
 'rougeL': 0.3960164115550142,
 'rougeLsum': 0.396040090323604}

## 评估完整的数据集

"diag-summary-training-results.csv"文件包含了所有模型结果的预填充列表，我们可以使用它来评估更大部分的数据。结果显示，在所有的ROUGE指标上都有显著的改善！

In [32]:
# 导入pandas库，用于数据处理和分析
import pandas as pd

# 使用pandas的read_csv函数读取CSV文件 "diag-summary-training-results.csv"，并将结果存储在results变量中
results = pd.read_csv("diag-summary-training-results.csv")

# 从results中提取 'original_model_summaries' 这一列的值，将其转换为numpy数组，并存储在 original_model_summaries 变量中
original_model_summaries = results['original_model_summaries'].values

# 从results中提取 'tuned_model_summaries' 这一列的值，将其转换为numpy数组，并存储在 tuned_model_summaries 变量中
tuned_model_summaries = results['tuned_model_summaries'].values

# 从results中提取 'human_baseline_summaries' 这一列的值，将其转换为numpy数组，并存储在 human_baseline_summaries 变量中
human_baseline_summaries = results['human_baseline_summaries'].values

In [33]:
# 使用 ROUGE 评估指标计算原始模型生成的摘要与人类生成的摘要之间的相似度
original_results = rouge.compute(
    # 预测的摘要，即原始模型生成的摘要
    predictions=original_model_summaries,
    # 参考的摘要，即人类生成的摘要，但只选取了与原始模型生成的摘要数量相同的部分
    references=human_baseline_summaries[0:len(original_model_summaries)],
    # 使用聚合器，这样可以计算多个摘要的平均 ROUGE 得分
    use_aggregator=True,
    # 使用词干处理，这样可以将单词的各种形态（如复数、过去式等）统一为其基本形式进行比较
    use_stemmer=True,
)

In [34]:
tuned_results = rouge.compute(
    predictions=tuned_model_summaries,
    references=human_baseline_summaries[0:len(tuned_model_summaries)],
    use_aggregator=True,
    use_stemmer=True,
)

In [35]:
original_results

{'rouge1': 0.2334158581572823,
 'rouge2': 0.07603964187010573,
 'rougeL': 0.20145520923859048,
 'rougeLsum': 0.20145899339006135}

In [36]:
tuned_results

{'rouge1': 0.42161291557556113,
 'rouge2': 0.18035380596301792,
 'rougeL': 0.3384439349963909,
 'rougeLsum': 0.33835653595561666}

In [37]:
# 计算微调模型与原始模型在各个 ROUGE 评估指标上的得分差异
improvement = (np.array(list(tuned_results.values())) - np.array(list(original_results.values())))

# 遍历微调模型的评估结果
for key, value in zip(tuned_results.keys(), improvement):
    # 打印每个 ROUGE 评估指标的名称（key）以及在此指标上的改进程度（value），并将改进程度转换为百分比的形式
    print(f'{key} absolute percentage improvement after instruct fine-tuning: {value*100:.2f}%')

rouge1 absolute percentage improvement after instruct fine-tuning: 18.82%
rouge2 absolute percentage improvement after instruct fine-tuning: 10.43%
rougeL absolute percentage improvement after instruct fine-tuning: 13.70%
rougeLsum absolute percentage improvement after instruct fine-tuning: 13.69%


# Release Resources

In [ ]:
%%html

<p><b>Shutting down your kernel for this notebook to release resources.</b></p>
<button class="sm-command-button" data-commandlinker-command="kernelmenu:shutdown" style="display:none;">Shutdown Kernel</button>
        
<script>
try {
    els = document.getElementsByClassName("sm-command-button");
    els[0].click();
}
catch(err) {
    // NoOp
}    
</script>